# Import training data

In [1]:
import pickle

data_file = './training_data.p'
with open(data_file, mode='rb') as f:
    data = pickle.load(f)
    car_image_names = data['car_image_names']
    not_car_image_names = data['not_car_image_names']

print(len(car_image_names))
print(len(not_car_image_names))

8792
8968


# Define helper functions

In [2]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.utils import shuffle
from lesson_functions import *

# Data preparation

## Extract features

In [3]:
color_space = 'HLS' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
spatial_size = (16, 16)
hist_bins = 32
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = 'ALL'
spatial_feat = True
hist_feat = True
hog_feat = True

t=time.time()
car_features = extract_features(car_image_names, color_space=color_space, spatial_size=spatial_size,
                                hist_bins=hist_bins, orient=orient,
                                pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel,
                                spatial_feat=spatial_feat, hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(not_car_image_names, color_space=color_space, spatial_size=spatial_size,
                                hist_bins=hist_bins, orient=orient,
                                pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel,
                                spatial_feat=spatial_feat, hist_feat=hist_feat, hog_feat=hog_feat)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract features...')

C:\ProgramData\Miniconda3\envs\carnd-term1\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


103.33 Seconds to extract features...


## Apply scaler

In [4]:
# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

## Split 80% 20% test set

In [5]:
# Split up data into randomized training and test sets
scaled_X, y = shuffle(scaled_X, y)

rand_state = np.random.randint(0, 100)

X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_state)

X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

print('Using:',orient,'orientations',pix_per_cell, 'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))

Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 6156


## Save processed features to pickle file

In [6]:
#pickle_file = './processed_features.p'
#
#try:
#    with open(pickle_file, 'wb') as pfile:
#        pickle.dump(
#            {
#                'X_train': X_train,
#                'y_train': y_train,
#                'X_test': X_test,
#                'y_test': y_test
#            },
#            pfile, pickle.HIGHEST_PROTOCOL)
#except Exception as e:
#    print('Unable to save data to', pickle_file, ':', e)
#    raise

# Training Classifier

## Parameter Tuning

In [7]:
#parameters = {'C':[0.001, 0.005, 0.01]}

#svr = LinearSVC()

# Use smaller sample size as GridSearchCV() takes a long time
#sample_size = 1000

#X_train, y_train = shuffle(X_train, y_train)

#t=time.time()
#clf = GridSearchCV(svr, parameters)
#clf.fit(X_train[0:sample_size], y_train[0:sample_size])
#t2 = time.time()

#print(round(t2-t, 2), 'Seconds to tune parameters...')
#print('Best parameters are: ', clf.best_params_)

Can't seem to get this function to produce sensible result. Using default C=1 to train.

## Training with processed features

In [8]:
# Use a linear SVC 
svc = LinearSVC()

# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()

n_predict = 100
print('My SVC predicts:     ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

17.37 Seconds to train SVC...
Test Accuracy of SVC =  0.9885
My SVC predicts:      [ 1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  1.  0.  1.
  1.  1.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  1.  0.  1.  0.
  1.  0.  0.  1.  1.  0.  1.  0.  1.  0.  1.  0.  0.  0.  1.  0.  0.  0.
  0.  1.  1.  0.  1.  1.  0.  0.  1.  1.  1.  0.  0.  1.  1.  1.  0.  1.
  1.  0.  0.  0.  0.  1.  1.  0.  1.  1.  1.  0.  1.  0.  0.  0.  0.  1.
  1.  0.  1.  1.  0.  0.  0.  0.  0.  0.]
For these 100 labels:  [ 1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  1.  0.  1.
  1.  1.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  1.  0.  1.  0.
  1.  0.  0.  1.  1.  0.  1.  0.  1.  0.  1.  0.  0.  0.  1.  0.  0.  0.
  0.  1.  1.  0.  1.  1.  0.  0.  1.  1.  1.  0.  0.  1.  1.  1.  0.  1.
  1.  0.  0.  0.  0.  1.  1.  0.  1.  1.  1.  0.  1.  0.  0.  0.  0.  1.
  1.  0.  1.  1.  0.  0.  0.  0.  0.  0.]
0.007 Seconds to predict 100 labels with SVC


## Save classifier to pickle file

In [9]:
pickle_file = './svc_pickle.p'

try:
    with open(pickle_file, 'wb') as pfile:
        pickle.dump(
            {
                'svc': svc,
                'X_scaler': X_scaler,
                'orient': orient,
                'pix_per_cell': pix_per_cell,
                'cell_per_block': cell_per_block,
                'spatial_size': spatial_size,
                'hist_bins': hist_bins,
                'color_space': color_space,
                'hog_channel': hog_channel,
                'spatial_feat': spatial_feat,
                'hist_feat': hist_feat,
                'hog_feat': hog_feat
            },
            pfile, pickle.HIGHEST_PROTOCOL)
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise